Second try at this implementation

1. Loads csv as pandas dataframe
2. Converts each line of the dataframe to tensor
3. Creates _MapDataset obj (same as obj recieved when loading sample data (TensorFlow2))
4. Preprocess data -> needs to convert to array and back to tensor to make operations -> CHANGE!! preprocess the csv before to simplify
5. Embed both towers and train the model
6. Obtain results

In [1]:
import tqdm as notebook_tqdm
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import pandas as pd
import functions as f

from typing import Dict, Text
import pprint

/home/pablo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-14 17:30:53.499590: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 17:30:53.499649: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 17:30:53.499684: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 17:30:53.506726: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is op

In [4]:
column_names = ['customer_id', 'article_id'] 
column_defaults = [tf.int64, tf.int64]

file_path = '../00 - Data/transactions/transactions_train_short2.csv'
batch_size = 32
num_epochs = 1

# Create the dataset
transactions = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    column_names=column_names,
    column_defaults=column_defaults,
    label_name='article_id',  
    header=True, 
    num_epochs=num_epochs
)

column_names = ['article_id'] 
column_defaults = [tf.int64]

file_path = '../00 - Data/articles/articles2.csv'
batch_size = 32
num_epochs = 1

# Create the dataset
articles = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    column_names=column_names,
    column_defaults=column_defaults,
    header=False, 
    num_epochs=num_epochs
)

In [25]:
articles

<_PrefetchDataset element_spec=OrderedDict([('article_id', TensorSpec(shape=(None,), dtype=tf.int64, name=None))])>

In [17]:
transactions

<_PrefetchDataset element_spec=(OrderedDict([('customer_id', TensorSpec(shape=(None,), dtype=tf.int64, name=None))]), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

<_MapDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [10]:

estimated_max_elements = 2.2e6 
embedding_dim = 32

# User tower (embedding for users)
user_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=int(estimated_max_elements), output_dim=embedding_dim, input_length=1)
])

# Article tower (embedding for articles)
article_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=int(105542), output_dim=embedding_dim, input_length=1)
])

In [13]:
# The `Task` objects has two purposes: (1) it computes the loss and (2)
    # keeps track of metrics.
eval_batch_size = 128

task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=articles.batch(eval_batch_size)
    )
)

In [31]:
class hmModel(tfrs.Model):
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      article_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.article_model = article_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features[0]['customer_id'])  # Accessing feature using integer index
 
    return self.task(user_embeddings, article_model)


In [ ]:
for element in articles.take(1):  # Print the first element for inspection
    print(element)

In [ ]:
model = hmModel(user_model,article_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(transactions.batch(4096), epochs=3)

In [ ]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
recommends.index_from_dataset(articles.batch(100).map(lambda title: (title, model.article_model(title))))
 
id_ = input('Enter the user_id: ')
_, titles = recommends(np.array([str(id_)]))
print('Top recommendation for user',id_,titles[0, :3])